<a href="https://colab.research.google.com/github/JasonLo/demo_haskins/blob/master/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo run OP model


In [2]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import pandas as pd

TensorFlow is already loaded. Please restart the runtime to change versions.
